# Úkol č. 2 - Segmentace zákazníků e-shopu

Jednou z důležitých aplikací shlukování je **segmentace zákazníků** (angl. **customer segmentation**). 

Předpokládejme, že máme následující obchodní údaje o prodejích (resp. nákupech z pohledu zákazníků):
TransactionID - ID nákupu,
CustomerID - ID zákazníka, 
Date - datum nákupu, 
Total - celková cena nákupu.

Chceme najít segmenty zákazníků, kteří se chovají podobně. K tomu je dobré informace z jednotlivých nákupů pro individuální zákazníky agregovat. Tj. získat pro každého zákazníka jeden řádek.

Populárním přístupem je **RFM**, což znamená:

- **R**ecency: Počet dnů od posledního nákupu (poslední datum v datasetu pro daného zákazníka)
- **F**requency: Počet nákupů. Občas se vynechávají zákazníci s jediným nákupem. Pro jednoduchost je zde ale necháme.
- **M**onetary: Celková suma, kterou daný zákazník utratil.

## Zdroj dat
Budeme pracovat s daty z jednoho (skoro) vymyšleného eshopu:

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
%matplotlib notebook

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

from scipy.cluster.hierarchy import dendrogram, linkage, fcluster

In [ ]:
df = pd.read_csv("eshop.csv", encoding="utf-8")
df["Date"] = pd.to_datetime(df["Date"], format="%m/%d/%Y")

## Pokyny k vypracování

**Základní body zadání**, za jejichž (poctivé) vypracování získáte **10 bodů**:
- Vytvořte `rfm` data frame, kde každý řádek odpovídá jednomu zákazníkovi a sloupce (příznaky) jsou uvedené výše.
- Pomocí algoritmu `K-means` proveďte shlukování. Nějakým způsobem také odhadněte nejlepší počet shluků (podrobně vysvětlete).
- Zabývejte se vlivem přeškálování dat (standardizace příznaků). Tj. určete, zda je přeškálování vhodné, a proveďte ho.
- Interpretujte jednotlivé shluky. Použijte získané shluky k odlišení "superstar" zákazníků (vysoká monetary, vysoká frequency a nízká recency) od nezajímavých  zákazníků (vysoká recency, nízká frequency, nízká monetary).

**Další body zadání** za případné další body  (můžete si vybrat, maximum bodů za úkol je každopádně 15 bodů):
- (až +5 bodů) Proveďte analýzu vytvořených shluků pomocí metody silhouette (https://en.wikipedia.org/wiki/Silhouette_(clustering)).
- (až +5 bodů) Zkuste provést to samé s modifikovanou verzí **RFM**, kde Recency = "maximum počtu měsíců od posledního nákupu a čísla 1", Frequency = "maximum počtu nákupů daného zákazníka v posledních 12 měsících (vztaženo k prosinci 2015) a čísla 1", Monetary = "Nejvyšší hodnota nákupu daného zákazníka". Porovnejte s původním přístupem.

## Poznámky k odevzdání

  * Řiďte se pokyny ze stránky https://courses.fit.cvut.cz/MI-PDM/homeworks/index.html.
  * Odevzdejte Jupyter Notebook.
  * Opravující Vám může umožnit úkol dodělat či opravit a získat tak další body. První verze je ale důležitá a bude-li odbytá, budete za to penalizováni


In [ ]:
def fancy_dendrogram(*args, **kwargs):
    max_d = kwargs.pop('max_d', None)
    if max_d and 'color_threshold' not in kwargs:
        kwargs['color_threshold'] = max_d
    annotate_above = kwargs.pop('annotate_above', 0)

    ddata = dendrogram(*args, **kwargs)

    if not kwargs.get('no_plot', False):
        plt.title('Dendrogram hierarchického shlukování (oříznutý)')
        plt.xlabel('index bodu nebo (velikost shluku)')
        plt.ylabel('vzdálenost')
        for i, d, c in zip(ddata['icoord'], ddata['dcoord'], ddata['color_list']):
            x = 0.5 * sum(i[1:3])
            y = d[1]
            if y > annotate_above:
                plt.plot(x, y, 'o', c=c)
                plt.annotate("%.3g" % y, (x, y), xytext=(0, -5),
                             textcoords='offset points',
                             va='top', ha='center')
        if max_d:
            plt.axhline(y=max_d, c='k')
    return ddata

In [ ]:
def plot3D(X, colors=None, show=True):
    plt.figure(figsize=(9, 9))
    ax = plt.axes(projection='3d')
    ax.scatter(X.iloc[:,0], X.iloc[:,1], X.iloc[:,2], c=colors)
    ax.set_xlabel(X.columns[0])
    ax.set_ylabel(X.columns[1])
    ax.set_zlabel(X.columns[2])
    if show:
        plt.show()
    return plt, ax

In [ ]:
def plot_dendrogram(data, p):
    plt.figure(figsize=(9, 9))
    plt.title('Dendrogram hierarchického shlukování')
    plt.xlabel('index bodu')
    plt.ylabel('vzdálenost')

    fancy_dendrogram(
        Z,
        truncate_mode='lastp',
        p=p,
        leaf_rotation=90.,
        leaf_font_size=12.,
        show_contracted=True
    )
    plt.show()

# Řešení
## RFM

In [ ]:
today = pd.to_datetime('today')
last_date = df.groupby("Customer ID")["Date"].max()

recency = (today - last_date).dt.days
frequency = df.groupby("Customer ID").size()
monetary = df.groupby("Customer ID")["Subtotal"].sum()

# X = rfm
X = pd.concat([recency, frequency, monetary], axis=1)
X.columns = ["Recency", "Frequency", "Monetary"]

plot3D(X);

## Hledání optimálního počtu shluků
### Dendrogram

In [ ]:
Z = linkage(X, 'single')
plot_dendrogram(Z, 15)

Podle dendrogramu začíná k oddělení shluků ve vzdálenosti 893. Touto hodnotou získáme 3 shluky. Toto rozdělení však není dobré, protože dva shluky budou každý obsahovat po jednom bodu a zbytek bodů bude obsažen v prvním shluku.

In [ ]:
max_d = 900
clusters = fcluster(Z, max_d, criterion='distance')

# Počet clusterů podle max_d
n_clusters = np.unique(clusters, return_counts=False).size
print("Počet clusterů pro max_d =", max_d, ": ", n_clusters)

# Vizualizace
plot3D(X, clusters);

## Účelová funkce

In [ ]:
max_clusters = 20
ix = np.zeros(max_clusters)
iy = np.zeros(max_clusters)
for k in range(ix.shape[0]):
    kmeans = KMeans(n_clusters=k + 1, random_state=1)
    kmeans.fit(X)
    iy[k] = kmeans.inertia_
    ix[k] = k+1
plt.figure(figsize=(7,7))
plt.xlabel('$k$')
plt.ylabel('Účelová funkce')
plt.plot(ix, iy, 'o-')
plt.show();

Z grafu účelové funkce není na první pohled hned jasné, kolik shluků je dobré vybrat. Dle mého uvážení je největší zlom v grafu pro hodnotu $k$ = 4.

## K-Means

Počet shluků jsem vybral na výsledku z grafu účelové funkce.

In [ ]:
n_clusters = 4
kmeans = KMeans(n_clusters=n_clusters, max_iter=1000)
kmeans.fit(X)
centroids = kmeans.cluster_centers_

plot, ax = plot3D(X, kmeans.labels_, show=False)

ax.set_xlabel(X.columns[0])
ax.set_ylabel(X.columns[1])
ax.set_zlabel(X.columns[2])
plot.show()

## Standardizace

Standardizace je zapotřebí, protože jednotlivé příznaky mají velké rozdíly mezi jejich hodnotami.

In [ ]:
scaled = X.values
# Standardizace
scaled = StandardScaler().fit_transform(scaled)
# Normalizace
# scaled = MinMaxScaler().fit_transform(scaled)
X_scaled = pd.DataFrame(scaled, columns=X.columns)

plot3D(X_scaled);

In [ ]:
Z = linkage(X_scaled, 'single')
plot_dendrogram(Z, p=10);

Zobrazení standardizovaného datasetu s využitím dendrogramu. Spodní graf má zobrazeny i uměle vybrány zajímavé zákazníky.

In [ ]:
max_d = 2
clusters = fcluster(Z, max_d, criterion='distance')

# Počet clusterů podle max_d
n_clusters = np.unique(clusters, return_counts=False).size
print("Počet clusterů pro max_d =", max_d, ": ", n_clusters)

# Vizualizace
plot3D(X_scaled, colors=clusters)

plot, ax = plot3D(X_scaled, colors=clusters, show=False);

# Zajímaví zákazníci
a = X_scaled
a = a[a["Recency"] <= 0]
a = a[a["Frequency"] >= 5]
a = a[a["Monetary"] >= 5]

ax.scatter(a.iloc[:,0], a.iloc[:,1], a.iloc[:,2], c="r")
plot.show();

In [ ]:
max_clusters = 20
ix = []
iy = []
for k in range(max_clusters):
    if k % 3 != 0: continue
    kmeans = KMeans(n_clusters=k + 1, random_state=1)
    kmeans.fit(X_scaled)
    iy.append(kmeans.inertia_)
    ix.append(k + 1)
plt.figure("Závislost účelové funkce na počtu clusterů", figsize=(7,7))
plt.xlabel('$k$')
plt.ylabel('Účelová funkce')
plt.plot(ix, iy, 'o-')
plt.show()

Zobrazení standardizovaného datasetu s využitím účelové funkce. Spodní graf má zobrazeny i uměle vybrány zajímavé zákazníky.

In [ ]:
n_clusters = 4
kmeans = KMeans(n_clusters=n_clusters, max_iter=1000)
kmeans.fit(X_scaled);
centroids = kmeans.cluster_centers_

plot3D(X_scaled, colors=kmeans.labels_)

plot, ax = plot3D(X_scaled, colors=kmeans.labels_, show=False)

# Zajímaví zákazníci
a = X_scaled
a = a[a["Recency"] <= 0]
a = a[a["Frequency"] >= 5]
a = a[a["Monetary"] >= 5]

ax.scatter(a.iloc[:,0], a.iloc[:,1], a.iloc[:,2], c="r")
plot.show()

Jednotlivé shluky se dle mého názoru nedají označit za správné shluky, jelikož jednotlivé body datasetu nebyly vytvořeny dle určitého pravděpodobnostního rozdělení. Jediný zajímavý shluk je ten, který má největší průnik s uměle vybraný zajímavý zákazníky. Tento shluk nejpravděpodobněji obsahuje zajímavé zákazníky.